In [1]:
import inspect
import fiftyone.utils.yolo as fouy

print(inspect.signature(fouy.add_yolo_labels))

(sample_collection, label_field, labels_path, classes, label_type='detections', include_missing=False)


In [ ]:
import fiftyone as fo
fo.launch_app(auto=True)

In [3]:
import torch
print(torch.cuda.get_device_name(0))
print(f"Total VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")


NVIDIA GeForce RTX 4060 Laptop GPU
Total VRAM: 8.00 GB


In [2]:
import pandas as pd

df = pd.read_parquet("../results/parsed_data/val_data.parquet")

df.head()




,image_name,split,label_id,category,x1,y1,x2,y2,width,height,scene,timeofday,weather,traffic_light_color,occluded,truncated
0,b1c66a42-6f7d68ca.jpg,val,0,traffic sign,1000.698742,281.992415,1040.626872,326.911560,1280,720,city street,daytime,overcast,none,False,False
1,b1c66a42-6f7d68ca.jpg,val,1,traffic sign,214.613695,172.190058,274.505889,229.586743,1280,720,city street,daytime,overcast,none,False,False
2,b1c66a42-6f7d68ca.jpg,val,2,traffic sign,797.314833,313.186265,829.756437,341.884608,1280,720,city street,daytime,overcast,none,False,False
3,b1c66a42-6f7d68ca.jpg,val,3,traffic sign,652.575363,303.204232,685.016968,315.681772,1280,720,city street,daytime,overcast,none,False,False
4,b1c66a42-6f7d68ca.jpg,val,4,traffic light,707.476543,311.938510,716.210821,328.159313,1280,720,city street,daytime,overcast,green,False,False


In [ ]:
import torch
from PIL import Image
import torchvision.transforms as T

model = torch.hub.load('hustvl/YOLOP', 'yolop', pretrained=True)  
model.eval()



In [42]:
from ultralytics import YOLO

model1 = YOLO("../runs/train/yolo11s_NoFreeze/weights/best.pt")
model2 = YOLO("../models/yolov8l_bdd.pt")



In [43]:
class_names_1 = model1.names
class_names_2 = model2.names

print(class_names_1)
print(class_names_2)

{0: 'person', 1: 'rider', 2: 'car', 3: 'bus', 4: 'truck', 5: 'bike', 6: 'motor', 7: 'traffic light', 8: 'traffic sign', 9: 'train'}
{0: 'pedestrian', 1: 'rider', 2: 'car', 3: 'truck', 4: 'bus', 5: 'train', 6: 'motorcycle', 7: 'bicycle', 8: 'traffic light', 9: 'traffic sign'}


In [13]:
metrics = model2.val(data="../configs/datasets/_bdd10_auto.yaml")

Ultralytics 8.3.162 🚀 Python-3.12.3 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 112 layers, 43,614,318 parameters, 0 gradients, 164.9 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1132.6±387.0 MB/s, size: 41.6 KB)


val: Scanning /home/anish/data/projects/autonomous-vision-bdd100k/data/labels/val.cache... 10000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10000/10000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 625/625 [03:14<00:00,  3.21it/s]


                   all      10000     185526      0.371      0.262      0.244      0.135
            pedestrian       3220      13262      0.604      0.574      0.592      0.294
                 rider        515        649      0.449      0.381       0.36      0.178
                   car       9879     102506      0.698      0.746      0.772      0.484
                 truck       1242       1597      0.124      0.357     0.0992      0.064
                   bus       2689       4245      0.305      0.142       0.23      0.146
                 train        578       1007          1          0          0          0
            motorcycle        334        452       0.52      0.283      0.318      0.146
               bicycle       5653      26885          0          0   1.53e-05   3.07e-06
         traffic light       8221      34908       0.01    0.00647     0.0706      0.036
          traffic sign         14         15   6.45e-05      0.133   1.79e-05   9.32e-06
Speed: 0.1ms preproce

In [44]:
metrics = model1.val(data="../configs/datasets/_bdd10_auto.yaml", save_json=True)

Ultralytics 8.3.162 🚀 Python-3.12.3 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)


YOLO11s summary (fused): 100 layers, 9,416,670 parameters, 0 gradients, 21.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 67.9±20.6 MB/s, size: 63.8 KB)


val: Scanning /home/anish/data/projects/autonomous-vision-bdd100k/data/labels/val.cache... 10000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10000/10000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 625/625 [01:41<00:00,  6.16it/s]


                   all      10000     185526      0.674      0.436      0.478      0.269
                person       3220      13262      0.678      0.513      0.574      0.281
                 rider        515        649      0.566      0.361      0.386      0.183
                   car       9879     102506      0.752       0.71      0.763      0.476
                   bus       1242       1597      0.606       0.52      0.563      0.432
                 truck       2689       4245      0.627      0.546      0.595      0.427
                  bike        578       1007       0.54      0.362      0.381      0.187
                 motor        334        452      0.597      0.279      0.342       0.17
         traffic light       5653      26885      0.687      0.512      0.565      0.212
          traffic sign       8221      34908       0.69      0.553      0.606      0.319
                 train         14         15          1          0          0          0
Speed: 0.1ms preproce

In [3]:
import fiftyone as fo
import fiftyone.utils.yolo as fouy

In [6]:
DATASET_NAME = "bdd10_auto_val"

if fo.dataset_exists(DATASET_NAME):
    fo.delete_dataset(DATASET_NAME)

In [8]:
DATASET_NAME = "bdd100k-val-ultralytics-pred"

if fo.dataset_exists(DATASET_NAME):
    fo.delete_dataset(DATASET_NAME)

In [30]:
if fo.dataset_exists("test-yolo-dataset"):
    print("Yes")

Yes


In [35]:
name = "test-yolo-dataset"
dataset_dir = "../data"

# Create the dataset
dataset = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=fo.types.YOLOv5Dataset, 
    name=name,
    overwrite=True,
)

In [32]:
# View summary info about the dataset
print(dataset)

# Print the first few samples in the dataset
print(dataset.head())

Name:        test-yolo-dataset
Media type:  image
Num samples: 10000
Persistent:  False
Tags:        []
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    ground_truth:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
[<Sample: {
    'id': '689c6415f943268c3325a6d6',
    'media_type': 'image',
    'filepath': '/home/anish/data/projects/autonomous-vision-bdd100k/data/images/val/b1c66a42-6f7d68ca.jpg',
    'tags': [],
    'metadata': None,
    'created_at': datetime.datetime(2025, 8, 13, 10, 8, 21, 621000),
    'last_modified_at': datetime.datetime(2025, 8, 13, 10, 8, 21, 621000),

In [36]:
dataset.count_sample_tags()  

{}

In [39]:
from ultralytics import YOLO
model = YOLO("../models/yolo11s_bdd.pt")
dataset.apply_model(model, label_field="YOLO")

In [ ]:
fo.launch_app(auto=True)

In [1]:
import fiftyone as fo
import fiftyone.core.labels as fol
import json
import os
from tqdm import tqdm

def load_bdd100k_json_to_fiftyone(
    json_path,
    images_dir,
    dataset_name="bdd100k",
    detection_classes=None,
):
    """
    Converts BDD100K JSON annotations to FiftyOne format with selected fields.
    
    Args:
        json_path (str): Path to BDD100K JSON file.
        images_dir (str): Directory containing the images.
        dataset_name (str): Name for the FiftyOne dataset.
        detection_classes (List[str], optional): Filter for object classes.
    
    Returns:
        fo.Dataset: FiftyOne dataset loaded with ground truth detections and attributes.
    """
    with open(json_path, "r") as f:
        data = json.load(f)

    samples = []

    for item in tqdm(data, desc=f"Loading {dataset_name}"):
        filepath = os.path.join(images_dir, item["name"])
        labels = item.get("labels", [])
        detections = []

        for obj in labels:
            cat = obj.get("category")
            box = obj.get("box2d")
            if not box or (detection_classes and cat not in detection_classes):
                continue

            x1, y1, x2, y2 = box["x1"], box["y1"], box["x2"], box["y2"]
            w, h = x2 - x1, y2 - y1

            attributes = obj.get("attributes", {})
            detection = fol.Detection(
                label=cat,
                bounding_box=[x1 / 1280, y1 / 720, w / 1280, h / 720],
                attributes={
                    "occluded": fol.Attribute(value=attributes.get("occluded", False)),
                    "truncated": fol.Attribute(value=attributes.get("truncated", False)),
                },
            )
            detections.append(detection)

        img_attrs = item.get("attributes", {})
        sample = fo.Sample(
            filepath=filepath,
            ground_truth=fol.Detections(detections=detections),
            scene=img_attrs.get("scene", "unknown"),
            weather=img_attrs.get("weather", "unknown"),
            timeofday=img_attrs.get("timeofday", "unknown"),
        )

        samples.append(sample)

    dataset = fo.Dataset(dataset_name, overwrite=True)
    dataset.add_samples(samples)
    dataset.compute_metadata()
    return dataset


In [3]:
from pycocotools.coco import COCO
import pandas as pd
import json

gt_path = "../data/coco_data/bdd100k_val_coco.json"
meta_path = "../data/parsed_data/val_data.parquet"

coco_gt = COCO(gt_path)
val_df = pd.read_parquet(meta_path)

with open("outputs/detect/val/predictions.json", "r") as f:
    preds = json.load(f)

print(" predictions image_id values:")
for p in preds[:5]:
    print(p['image_id'], type(p['image_id']))


print(" COCO GT image entries:")
for img in coco_gt.dataset['images'][:5]:
    print(img['id'], img['file_name'], type(img['id']))

loading annotations into memory...
Done (t=0.96s)
creating index...
index created!
 predictions image_id values:
fe1f55fa-19ba3600 <class 'str'>
fe1f55fa-19ba3600 <class 'str'>
fe1f55fa-19ba3600 <class 'str'>
fe1f55fa-19ba3600 <class 'str'>
fe1f55fa-19ba3600 <class 'str'>
 COCO GT image entries:
1 b1c66a42-6f7d68ca.jpg <class 'int'>
2 b1c81faa-3df17267.jpg <class 'int'>
3 b1c81faa-c80764c5.jpg <class 'int'>
4 b1c9c847-3bda4659.jpg <class 'int'>
5 b1ca2e5d-84cf9134.jpg <class 'int'>


In [4]:
file_to_id = {img['file_name'].replace('.jpg', ''): img['id'] for img in coco_gt.dataset['images']}

# Update predictions to use numeric IDs
for p in preds:
    p['image_id'] = file_to_id[p['image_id']]

# Now we can load into COCOeval
coco_dt = coco_gt.loadRes(preds)

Loading and preparing results...
DONE (t=8.13s)
creating index...
index created!


In [5]:
print(set(type(p['image_id']) for p in preds)) 

pred_ids = {p['image_id'] for p in preds}
gt_ids = set(coco_gt.getImgIds())
print(len(pred_ids & gt_ids) == len(pred_ids))


{<class 'int'>}
True
